In [1]:
import gmaps
import numpy as np
import pandas as pd
import os
import scipy.stats as st
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import requests
# # Google developer API key
from config import gkey

# # Configure gmaps
# gmaps.configure(api_key=gkey)

# Create aiport dataframe

path = pd.read_csv('../01._Prospective_Project_Data/1._72199_158097_compressed_indeed_job_dataset.csv/indeed_job_dataset_V2_CSV.csv')
df = pd.DataFrame(path)
df
# airport_df = pd.read_csv('../Resources/Airport_Output.csv')
df.fillna(0) 
df


FileNotFoundError: [Errno 2] File ../01._Prospective_Project_Data/1._72199_158097_compressed_indeed_job_dataset.csv/indeed_job_dataset_V2_CSV.csv does not exist: '../01._Prospective_Project_Data/1._72199_158097_compressed_indeed_job_dataset.csv/indeed_job_dataset_V2_CSV.csv'

In [ ]:

df_cleaned = df.drop(columns=df.columns[(df['No_of_Skills'] == '0').any()])
df_cleaned = df.dropna(subset=['No_of_Skills'])
df_cleaned['No_of_Skills'].dtype

In [ ]:
#stats summary
average=df_cleaned.groupby(['Job_Type']).mean()['Salary_Index']
median=df_cleaned.groupby(['Job_Type']).median()['Salary_Index']
variance=df_cleaned.groupby(['Job_Type']).var()['Salary_Index']
standard_deviation=df_cleaned.groupby(['Job_Type']).std()['Salary_Index']
sem=df_cleaned.groupby(['Job_Type']).sem()['Salary_Index']

# Created a DF
stats_summary=pd.DataFrame({"Mean Salary Index": average,
                           "Median Salary Index": median,
                           "Salary Index Variance":variance,
                           "Salary Index Std. Dev.":standard_deviation,
                           "Salary Index Std. Err.":sem
                          })



In [ ]:
stats_summary

In [ ]:
df = df_cleaned['Salary_Index'].astype(int)

In [ ]:
#create DF by industry type, salary, and job title 
industry_df=df_cleaned[['Job_Title',\
                       'Queried_Salary',\
                       'Consulting and Business Services',\
                       'Internet and Software',\
                       'Banks and Financial Services',\
                       'Health Care',\
                       'Insurance',\
                       'Other_industries']]
industry_df


In [ ]:
#filter by highest pay 
high_pay_industry=industry_df[industry_df['Queried_Salary']== '>160000']
high_pay_industry


In [ ]:
#calculate total nums of industry for each job title
#consulting
consulting_tot=high_pay_industry['Consulting and Business Services'].sum(axis = 0, skipna = True)

#internet & software
internet_tot=high_pay_industry['Internet and Software'].sum(axis = 0, skipna = True)

#banks
banks_tot=high_pay_industry['Banks and Financial Services'].sum(axis = 0, skipna = True)

#health
health_tot=high_pay_industry['Health Care'].sum(axis = 0, skipna = True)

#insurance
insurance_tot=high_pay_industry['Insurance'].sum(axis = 0, skipna = True)

#other industry
other_tot=high_pay_industry['Other_industries'].sum(axis = 0, skipna = True)

print(f'total consulting = {consulting_tot}')
print(f'total for internet= {internet_tot}')
print(f'total for banks and finance= {banks_tot}')
print(f'total for health= {health_tot}')
print(f'total for insurance= {insurance_tot}')
print(f'total for other industries= {other_tot}')


In [ ]:
total_industry_count=consulting_tot+internet_tot+banks_tot+health_tot+insurance_tot+other_tot
total_industry_count

In [ ]:
#percentage by industry
consult_percent="{:.0%}".format(consulting_tot/total_industry_count)

internet_percent="{:.0%}".format(internet_tot/total_industry_count)

banks_percent="{:.0%}".format(banks_tot/total_industry_count)

health_percent="{:.0%}".format(health_tot/total_industry_count)

insurance_percent="{:.0%}".format(insurance_tot/total_industry_count)

other_percent="{:.0%}".format(other_tot/total_industry_count)


# percentage_summary=pd.DataFrame({"Count of Consulting Industry": consulting_tot,
#                            "Count of Internet Industry":internet_tot,
#                            "Count of Banks":banks_tot,
#                            "Count of Health":health_tot,
#                            "Count of Insurance":insurance_tot,
#                             "Count of Other":other_tot },index=[0])
# percentage_summary
data={'Industries':["Consulting and Business Services",\
                    "Internet and Software",\
                    "Banks and Financial Services",\
                    "Health Care",\
                    "Insurance","\
                    Other Industries"],'Totals':[consulting_tot,internet_tot,banks_tot,health_tot,insurance_tot,other_tot]}
summary_df = pd.DataFrame(data)
summary_df

In [ ]:
subset=summary_df.set_index('Industries')
subset
# plot = summary_df.plot.pie(subplots=True, figsize=(11, 6))

In [ ]:
subset.plot.pie(y='Totals',figsize=(6, 6),autopct='%1.1f%%', startangle=90,pctdistance=1.1, labeldistance=1.2)
ax=plt.subplot()
ax.legend(loc='upper right',bbox_to_anchor=(2.5,1.00))
plt.show()

In [ ]:
# total_no_cities=df["Location"].unique()
# total_no_cities
city_index = (df_cleaned[ df_cleaned['Location'] == 'REMOTE' ].index)
df_cleaned.drop(city_index, inplace=True)

city_index = (df_cleaned[ df_cleaned['Location'] == 'USA' ].index)
df_cleaned.drop(city_index, inplace=True)

In [ ]:
city_index2 = df_cleaned[ df_cleaned['Location'] == 'nan' ].index
df_cleaned.drop(city_index2, inplace=True)


In [ ]:
df_cleaned.dropna(subset=['Location'], inplace=True)


# Jobs By Company

In [ ]:
for col in df_cleaned.columns: 
    print(col) 

In [ ]:

#find company with most job listings
num_company=df_cleaned['Company'].nunique()
print(f'There are a total of {num_company} job openings')

most_job_openings= df_cleaned.groupby(['Company'])['Job_Type'].count()
top_company=pd.DataFrame(most_job_openings.nlargest(20))


plot=top_company.plot(kind='bar', align="center",figsize = (15,7))
plt.title("Companies with Highest Job Listings")
plt.show()



In [ ]:
job_count_city=df_cleaned.groupby(['Location'])['Company'].count()
state_df=pd.DataFrame(job_count_city.nlargest(50))
state_df


In [ ]:
plot=state_df.plot(kind='bar', align="center",figsize = (15,7))
plt.title('Job openings by State')
plt.show()

# City Analysis

In [ ]:
industry_df=pd.DataFrame(df_cleaned.loc[:,['Location',\
                                    'Consulting and Business Services',\
                                    'Internet and Software',\
                                    'Banks and Financial Services',\
                                    'Health Care',\
                                   'Insurance',\
                                    'Company']])
industry_df.dropna()

industry_df.head()


In [ ]:
state_df["Lat"] = ""
state_df["Lng"] = ""
state_df.groupby(['Location'])
state_bylocation=pd.DataFrame(state_df)
state_bylocation.reset_index(inplace=True)
state_bylocation

In [ ]:
params = {"key": gkey}
max_rows=200
# Loop through the location and run a lat/long search for each city
for index, row in state_bylocation.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    state = row['Location']
    

    # update address key value
    params['address'] = f"{state}"

#     # make request
    state_lat_lng = requests.get(base_url, params=params)

#     # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
#     print(state_lat_lng.url)
    
# #     # convert to json
    state_lat_lng = state_lat_lng.json()
#     print(state_lat_lng['results'][0]['geometry']['location'])
#     time.sleep(1.00)

    state_bylocation.loc[index, "Lat"] = state_lat_lng["results"][0]['geometry']["location"]["lat"]
    state_bylocation.loc[index,'Lng']= state_lat_lng["results"][0]['geometry']["location"]["lng"]
    if index >max_rows and max_rows>0:
        break
state_bylocation.head()


In [ ]:
locations=state_bylocation[['Lat','Lng']]
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=state_bylocation['Company'], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
state_template="""<dl><dd>{Location}</dd></dl>"""
state_info=[state_template.format(**row) for index,row in state_bylocation.iterrows()]
# print(state_info)
marker_layer=gmaps.marker_layer(locations,info_box_content= state_info)
fig.add_layer(marker_layer)


fig

In [ ]:
target_state = industry_df.groupby(['Location'])['Company']

params = {"address": state, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"


# Run request
response = requests.get(base_url, params=params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

# Convert to JSON
response = response.json()

# Extract lat/lng
lat = response["results"][0]["geometry"]["location"]["lat"]
lng = response["results"][0]["geometry"]["location"]["lng"]

# Print results
print('Hi')
print(f"{target_state}: {lat}, {lng}")

In [ ]:
# city_count=df_cleaned['Location'].value_counts()
# city_name=df_cleaned['Location']
# city_df=pd.DataFrame({'Location':city_count,'Total Jobs': city_count})
# city=len(df_cleaned['Location'])
# city_df

In [ ]:
state_ratio=df_cleaned['Location'].value_counts(normalize=True)*100
statedata_df=pd.DataFrame(state_ratio)
statedata_df.reset_index(level=0, inplace=True)
clean_state=statedata_df.rename(columns={"index": "Location", "Location": "Total Jobs %"})

clean_state.head()


In [ ]:
state_plot=statedata_df.plot.bar(x='index', y='Location',rot=90)


In [ ]:

clean_state=statedata_df.rename(columns={"index": "Location", "Location": "Total Jobs"})
# city_percent=clean_city.style.format({'Total Jobs': "{:.2%}".format})


In [ ]:
clean_state